# Hardest negative contrastive loss

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import torch

In [28]:
n_points = 4
margin_pos = 0.1
margin_neg = 1.4
feat_dim = 4

In [29]:
feat2d = torch.randn(1, n_points, feat_dim)
feat3d = torch.randn(1, n_points, feat_dim)

In [30]:
eps = 1e-6
feat2d_norm = feat2d / (torch.norm(feat2d, p=2, dim=1, keepdim=True).detach() + eps)
feat3d_norm = feat3d / (torch.norm(feat3d, p=2, dim=1, keepdim=True).detach() + eps)

In [31]:
feat2d_norm

tensor([[[-0.5286,  0.9696,  0.4711,  0.2617],
         [-0.5332,  0.0247, -0.6069, -0.3971],
         [-0.0604,  0.2373, -0.4226, -0.5069],
         [-0.6577, -0.0536, -0.4807,  0.7189]]])

In [32]:
feat3d_norm

tensor([[[-0.2904, -0.0780, -0.6476,  0.2456],
         [-0.6682,  0.2467, -0.4937, -0.1344],
         [ 0.6682,  0.7578, -0.0304,  0.9531],
         [ 0.1509, -0.5990,  0.5796,  0.1148]]])

In [35]:
dists = torch.cdist(feat2d_norm, feat3d_norm).squeeze()
dists

tensor([[1.5512, 1.2766, 1.4856, 1.7192],
        [0.6958, 0.3865, 2.0337, 1.5896],
        [0.8770, 0.7164, 1.7571, 1.4612],
        [0.6225, 0.9048, 1.6352, 1.5622]])

In [36]:
loss_pos = ((dists.diagonal() - margin_pos)**2).mean()

In [39]:
# set diagonal to inf, then find the closest negatives
ind = torch.arange(dists.shape[0])
dists[ind, ind] = float('inf')
dists

tensor([[   inf, 1.2766, 1.4856, 1.7192],
        [0.6958,    inf, 2.0337, 1.5896],
        [0.8770, 0.7164,    inf, 1.4612],
        [0.6225, 0.9048, 1.6352,    inf]])

In [43]:
dists.min(axis=1)[0], dists.min(axis=0)[0]

(tensor([1.2766, 0.6958, 0.7164, 0.6225]),
 tensor([0.6225, 0.7164, 1.4856, 1.4612]))

In [45]:
loss_neg = 0.5*((margin_neg - dists.min(axis=1)[0])**2 + (margin_neg - dists.min(axis=0)[0])**2).mean()
loss_neg

tensor(0.3332)

In [52]:
loss = loss_pos + loss_neg
loss

tensor(2.1012)